# <center> Mesh Normals </center>

---



# Setup

## Prevent colab from disconnecting

```
function ClickConnect(){
  console.log("Connnect Clicked - Start"); 
  document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
  console.log("Connnect Clicked - End"); 
};
setInterval(ClickConnect, 60000)
```

## Installation

### Trimesh

In [1]:
!pip install trimesh --upgrade

     |████████████████████████████████| 634 kB 9.5 MB/s 


### Pytorch

In [2]:
!pip install torch

In [3]:
import torch

TORCH_VERSION = torch.__version__[:5]
CUDA_VERSION = torch.version.cuda.replace('.','')
print(f'Running torch version {TORCH_VERSION}, with CUDA version {CUDA_VERSION}')

Running torch version 1.9.0, with CUDA version 102


## Imports

In [4]:
# deep learning
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim
import torch.utils.data as data
torch.set_printoptions(threshold=10000)

# meshes and graphs
import trimesh
import networkx as nx

# data
import pandas as pd
import numpy as np
import scipy
from scipy.sparse import linalg
from scipy.sparse import coo_matrix
from scipy.linalg import eig
from scipy.linalg import null_space

# plots
import matplotlib.pyplot as plt
import seaborn as sns

# i/o
import dill
import json

# misc
import os
import math
import random
from typing import *
from pprint import *
from tqdm.notebook import tqdm

## Google Drive

In [5]:
from google.colab import drive
GDRIVE_HOME = '/content/drive'

drive.mount(GDRIVE_HOME, force_remount=True)

Mounted at /content/drive


## Configuration

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [7]:
PROJECT_HOME = os.path.join(GDRIVE_HOME, 'MyDrive/TSP-SC/code/mesh normals')
data_folder = os.path.join(PROJECT_HOME, f'data')

## Run settings

In [8]:
USE_NOISY = True

In [9]:
use_dummy_data = False

# Functions

## Utilities

In [17]:
def print_number_of_parameters(network):
    num_params = 0
    for param in network.parameters():
        p = np.array(param.shape, dtype=int).prod()
        num_params += p
    print("Total number of parameters: %d" %(num_params))


def coo2tensor(A):
    """
        Converts a sparse matrix in COOrdinate format to torch tensor.
    """
    assert scipy.sparse.isspmatrix_coo(A)
    idxs = torch.LongTensor(np.vstack((A.row, A.col)))
    vals = torch.FloatTensor(A.data)
    return torch.sparse_coo_tensor(idxs, vals, size=A.shape, requires_grad=False)


def normalize(L, half_interval=False):
    """
        Returns the laplacian normalized by the largest eigenvalue
    """
    assert scipy.sparse.isspmatrix(L)

    # L is squared
    M = L.shape[0]
    assert M == L.shape[1]

    # take the first eigenvalue of the Laplacian, i.e. the largest
    largest_eigenvalue = linalg.eigsh(L, k=1, which="LM", return_eigenvectors = False)[0]   

    L_normalized = L.copy()

    if half_interval:
        L_normalized *= 1.0/largest_eigenvalue
    else:
        L_normalized *= 2.0/largest_eigenvalue
        L_normalized.setdiag(L_normalized.diagonal(0) - np.ones(M), 0)

    return L_normalized

# Models

## MySCNN

\begin{align}
    s_{sol} &= F_k^{-1}(\varphi_{W_{sol}}) *_k s = \sum_{i=0}^{N} \left( (W_{sol})_i U_{sol} diag(\Lambda_{sol}^i)U_{sol}^\top \right) s = \sum_{i=0}^{N} \left( (W_{sol})_i \left( U diag(\Lambda) U^\top \right)^i \right) s \\
    &= \sum_{i=0}^{n} \left((W_{sol})_i (L_k)_{sol}^i \right)s 
    = \sum_{i=0}^{n} \left((W_{sol})_i (L_k)_{sol}^i \right)s = \sum_{i=0}^{n} \left((W_{sol})_i (B_k^\top B_k)^i \right)s 
\end{align}

### Framework

The Chebyshev polynomial $T_k(x)$ of order $k$ may be computed by the stable recurrence relation 
$$
    T_k(x) = 2x T_{k-1}(x) - T_{k-2}(x)
$$
with $T_0 = 1$ and $T_1 = x$.

A filter can be parametrized as the truncated expansion
$$
    g_\theta(\Lambda) = \sum_{k=0}^{K-1} \theta_k T_k (\tilde{\Lambda})
$$
where the parameter $\theta \in \mathbb{R}^k$ is a vector of Chebyshev coefficients and $T_k(\tilde{\Lambda}) \in \mathbb{R}^{n \times n}$ is the Chebyshev polynomial of order $k$ evaluated at $\tilde{\Lambda} = 2 \Lambda/ \lambda_{max} - I_{n}$, a diagonal matrix of scaled eigenvalues that lie in $[-1, 1]$. The filtering operation can then be written as 
$$
    y = g_\theta(L)x = \sum_{k=0}^{K-1} \theta_k T_{k}(\tilde{L})x
$$
where $T_{k}(\tilde{L})$ is the Chebyshev polynomial of order $k$ evaluated at the scaled Laplacian $\tilde{L} = 2L/\lambda_{max} - I_{n}$. Denoting $\bar{x}_{k} = T_{k}(\tilde{L})x$, we can use the recurrence relation to compute 
$$
    \bar{x}_{k} = 2 \tilde{L}\bar{x}_{k-1} - \bar{x}_{k-2}
$$
with 
*   $\bar{x_0} = x$
*   $\bar{x}_1 = \tilde{L}x$




In [18]:
def my_assemble(filter_size, L, x):
    """
    parameters:
        filter_size: Chebyshev filter size
        L: Laplacian (num_simplices, num_simplices)
        x: input (batch_size, C_in, num_simplices)
    """
    
    (C_in, num_simplices) = x.shape

    assert L.shape[0] == num_simplices
    assert L.shape[0] == L.shape[1] # L is a square matrix
    assert filter_size > 0
    x = x.to(torch.float32)
    X = []
    # for each channel
    for c_in in range(0, C_in):
        bar_X = []
        bar_X_0 = x[c_in, :].unsqueeze(1) # \bar{x}_0 = x
        bar_X.append(bar_X_0)  
        
        # Chebyshev recursion
        if filter_size > 1:
            bar_X_1 = L @ bar_X[0] # \bar{x}_1 = L x
            bar_X.append(bar_X_1)

            for k in range(2, filter_size): 
                bar_X.append(2*(L @ bar_X[k-1]) - bar_X[k-2]) # \bar{x}_k = 2 L \bar{x}_{k-1} - \bar{x}_{k-2}

        # (num_simplices, filter_size)
        bar_X = torch.cat(bar_X, 1)
        assert bar_X.shape == (num_simplices, filter_size)
        X.append(bar_X.unsqueeze(0))
    
    # (channels_in, num_simplices, filter_size)
    X = torch.cat(X, 0)
    assert X.shape == (C_in, num_simplices, filter_size)

    return X

#### Convolution

In [19]:
class MySimplicialConvolution(nn.Module):
    def __init__(self, filter_size, C_in, C_out, enable_bias=True, variance=1.0):
        """
        Convolution for simplices of a fixed dimension
        """
        super().__init__()
        
        assert C_in > 0
        assert C_out > 0 
        assert filter_size > 0 
        
        self.C_in = C_in
        self.C_out = C_out
        self.filter_size = filter_size
        self.enable_bias = enable_bias

        self.theta = nn.parameter.Parameter(variance*torch.randn((self.C_out, self.C_in, self.filter_size)))
        self.bias = nn.parameter.Parameter(torch.zeros((self.C_out, 1))) if self.enable_bias else 0.0

    def forward(self, L, x):
        (channels_in, num_simplices) = x.shape

        assert channels_in == self.C_in

        X = my_assemble(self.filter_size, L, x)

        # X ~ channels_in, num_simplices, filter_size
        # theta ~ channels_out, channels_in, filter_size 
        # y ~ channels_out, num_simplices

        y = torch.einsum("imk, oik -> om", (X, self.theta)) + self.bias

        assert y.shape == (self.C_out, num_simplices)

        return y


### Network definition

In [20]:
class MySCNN(nn.Module):
    def __init__(self, filter_size, colors):
        super().__init__()

        assert colors > 0
        self.colors = colors

        num_filters = 5
        variance = 0.01 
        self.num_layers = 5
        self.num_dims = 3

        self.activaction = nn.LeakyReLU()

        self.C = nn.ModuleDict({ f'l{i}': nn.ModuleDict()  for i in range(1, self.num_layers+1) })
        self.aggr = nn.ModuleDict({ f'l{i}': nn.ModuleDict()  for i in range(1, self.num_layers+1) })

        # layer 1 
        self.C['l1']['d0'] = MySimplicialConvolution(filter_size, C_in=self.colors, C_out=self.colors*num_filters, variance=variance)

        # layer 2 
        self.C['l2']['d0'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l2']['d1'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.aggr['l2']['d1'] = nn.Sequential(
            nn.Linear(2*self.colors*num_filters, self.colors*num_filters),
            nn.ReLU()
        )

        # layer 3 
        self.C['l3']['d0'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l3']['d1'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l3']['d2'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.aggr['l3']['d1'] = nn.Sequential(
            nn.Linear(2*self.colors*num_filters, self.colors*num_filters),
            nn.ReLU()
        )
        self.aggr['l3']['d2'] = nn.Sequential(
            nn.Linear(2*self.colors*num_filters, self.colors*num_filters),
            nn.ReLU()
        )
        
        # layer 4  
        self.C['l4']['d0'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l4']['d1'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l4']['d2'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.aggr['l4']['d1'] = nn.Sequential(
            nn.Linear(2*self.colors*num_filters, self.colors*num_filters),
            nn.ReLU()
        )
        self.aggr['l4']['d2'] = nn.Sequential(
            nn.Linear(2*self.colors*num_filters, self.colors*num_filters),
            nn.ReLU()
        )

        # layer 5  
        self.C['l5']['d0'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l5']['d1'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.C['l5']['d2'] = MySimplicialConvolution(filter_size, C_in=self.colors*num_filters, C_out=self.colors*num_filters, variance=variance)
        self.aggr['l5']['d1'] = nn.Sequential(
            nn.Linear(2*self.colors*num_filters, self.colors*num_filters),
            nn.ReLU()
        )

        self.last_aggregator = nn.Linear(2*self.colors*num_filters, self.colors)


    def forward(self, xs, components, Bs, Bts):
        """
        parameters:
            xs: inputs
        """

        layers = range(self.num_layers+1)
        dims = range(self.num_dims)
        L = components['lap']

        ###### layer 1 ######

        # S0 = conv(S0)
        # (num_filters x num_dims, num_nodes) 
        S0 = self.C['l1']['d0'](L[0], xs[0])
        S0 = self.activaction(S0)

        # S1 = lift(S0)        
        # (num_edges, num_filters * c_in)
        S0_lifted = self.lift(Bts[0], S0)
        S1 = S0_lifted

        ###### layer 2 ######
        
        # S0 = conv(S0)
        # (num_filters * num_dims, num_nodes)
        S0 = self.C['l2']['d0'](L[0], S0)
        S0 = self.activaction(S0)

        # (num_edges, num_filters * c_in)
        S0_lifted = self.lift(Bts[0], S0)

        # (num_filters * c_in, num_edges)
        S1_conv = self.C['l2']['d1'](L[1], S1.transpose(1, 0))
        S1_conv = self.activaction(S1_conv)

        S1_concat = torch.cat((S0_lifted, S1_conv.transpose(1, 0)), dim=1)
        
        S1 = self.aggr['l2']['d1'](S1_concat)

        # S2 = lift(S1)
        S2 = Bts[1] @ S1

        ###### layer 3 ######

        # S0 = conv(S0)
        S0 = self.C['l3']['d0'](L[0], S0)
        S0 = self.activaction(S0)

        # (num_edges, num_filters * c_in)
        S0_lifted = self.lift(Bts[0], S0)

        # (num_filters * c_in, num_edges)
        S1_conv = self.C['l3']['d1'](L[1], S1.transpose(1, 0))
        S1_conv = self.activaction(S1_conv)

        # (2 * num_filters * c_in, num_edges)
        S1_concat = torch.cat((S0_lifted, S1_conv.transpose(1, 0)), dim=1)
        
        S1 = self.aggr['l3']['d1'](S1_concat)
        
        # (num_edges, num_filters * c_in)
        S1_lifted = Bts[1] @ S1

        # (num_filters * c_in, num_edges)
        S2_conv = self.C['l3']['d2'](L[2], S2.transpose(1, 0))
        S2_conv = self.activaction(S2_conv)

        S2_concat = torch.cat((S1_lifted, S2_conv.transpose(1, 0)), dim=1)
        
        S2 = self.aggr['l3']['d2'](S2_concat)

        ###### layer 4 ######

        # S0 = conv(S0)
        S0 = self.C['l4']['d0'](L[0], S0)
        S0 = self.activaction(S0)

        # (num_edges, num_filters x c_in)
        S0_lifted = self.lift(Bts[0], S0)

        # (num_filters x c_in, num_edges)
        S1_conv = self.C['l4']['d1'](L[1], S1.transpose(1, 0))
        S1_conv = self.activaction(S1_conv)

        S1_concat = torch.cat((S0_lifted, S1_conv.transpose(1, 0)), dim=1)
        
        S1 = self.aggr['l4']['d1'](S1_concat)

        # (num_edges, num_filters x c_in)
        S1_lifted = Bts[1] @ S1

        # (num_filters x c_in, num_edges)
        S2_conv = self.C['l4']['d2'](L[2], S2.transpose(1, 0))
        S2_conv = self.activaction(S2_conv)

        S2_concat = torch.cat((S1_lifted, S2_conv.transpose(1, 0)), dim=1)
        
        S2 = self.aggr['l4']['d2'](S2_concat)

        ###### layer 5 ######

        # S0 = conv(S0)
        S0 = self.C['l4']['d0'](L[0], S0)
        S0 = self.activaction(S0)

        # (num_edges, num_filters x c_in)
        S0_lifted = self.lift(Bts[0], S0)

        # (num_filters x c_in, num_edges)
        S1_conv = self.C['l4']['d1'](L[1], S1.transpose(1, 0))
        S1_conv = self.activaction(S1_conv)

        S1_concat = torch.cat((S0_lifted, S1_conv.transpose(1, 0)), dim=1)
        
        S1 = self.aggr['l5']['d1'](S1_concat)

        # (num_edges, num_filters x c_in)
        S1_lifted = Bts[1] @ S1

        # (num_filters x c_in, num_edges)
        S2_conv = self.C['l4']['d2'](L[2], S2.transpose(1, 0))

        S2_concat = torch.cat((S1_lifted, S2_conv.transpose(1, 0)), dim=1)
        
        S2 = self.last_aggregator(S2_concat)

        return S2

    def lift(self, B, S):
        return B @ S.transpose(1, 0)

## Linear baseline

In [21]:
class LinearBaseline(nn.Module):
    def __init__(self, num_nodes, num_triangles):
        super().__init__()
        self.num_nodes = num_nodes
        self.num_triangles = num_triangles

        self.node_to_triangles = nn.Linear(self.num_nodes, self.num_triangles, dtype=float)
        self.linear = nn.Linear(self.num_triangles, self.num_triangles, dtype=float)


    def forward(self, X):
        """
        """
        triangle_features = self.node_to_triangles(X)
        
        triangle_features = nn.ReLU()(triangle_features)

        output = self.linear(triangle_features)
        
        return output.transpose(1, 0)

## Topology-aware baseline

In [110]:
from scipy.sparse import csr_matrix


def create_node_triangle_adj_matrix(nodes, triangles):
    """
        nodes: tensor (num_nodes, 3)
        triangles: tensor (num_triangles, 3)
    """
    
    i = np.arange(triangles.shape[0])
    i = np.concatenate((i,i,i), 0)

    j = triangles.reshape(-1)

    adj = csr_matrix((np.ones_like(j), (i, j)), shape=(triangles.shape[0], nodes.shape[0])) #FxV sparse matrix

    return torch.tensor(adj.todense(), dtype=torch.float64).to(device)

In [111]:
stacked_triangles = np.stack(triangles)
node_triangle_adj = create_node_triangle_adj_matrix(original_positions, stacked_triangles)

In [138]:
class TopologyAwareBaseline(nn.Module):
    def __init__(self, num_nodes, num_triangles, node_triangle_adj):
        """
            node_triangle_adj: tensor (num_triangles, num_nodes)
        """
        super().__init__()
        self.num_nodes = num_nodes
        self.num_triangles = num_triangles
        self.node_triangle_adj = node_triangle_adj.transpose(1, 0)
        # self.node_triangle_adj.requires_grad = False

        self.linear = nn.Linear(num_triangles, num_triangles, dtype=torch.float64)
    
    def forward(self, X):
        """
            X: tensor (3, num_nodes)
        """
        triangle_features =  X @ self.node_triangle_adj
        
        # (num_triangles, num_nodes)
        triangle_features = nn.ReLU()(triangle_features)

        output = self.linear(triangle_features)
        
        return output.transpose(1, 0)

# Data loading

In [24]:
def load_dict(path):
    keys_path = path + '_keys'
    values_path = path + '_values.npy'

    dictionary = {}

    with open(keys_path, 'rb') as f:
        keys = dill.load(f)

    values = np.load(values_path)

    for k,v in zip(keys, values):
        dictionary[k] = v

    return dictionary

In [25]:
prefix = 'dummy_' if use_dummy_data else ''

laplacian_path = f'{data_folder}/{prefix}laplacians.npy'
boundaries_path = f'{data_folder}/{prefix}boundaries.npy'
positions_path = f'{data_folder}/{prefix}positions'
normals_path = f'{data_folder}/{prefix}normals'
triangles_path = f'{data_folder}/{prefix}triangles.npy'
original_positions_path = f'{data_folder}/{prefix}original_positions'
noisy_positions_path = f'{data_folder}/{prefix}noisy_positions'

In [26]:
laplacians = np.load(laplacian_path, allow_pickle=True)
print(len(laplacians))

3


In [27]:
boundaries = np.load(boundaries_path, allow_pickle=True)
print(len(boundaries))

2


In [28]:
triangles = np.load(triangles_path, allow_pickle=True)
print(len(triangles))

10688


In [29]:
original_positions = np.load(original_positions_path, allow_pickle=True)

In [30]:
signals = {0: {}, 1: {}, 2: {}}

signals[0] = load_dict(positions_path)

signals[2] = load_dict(normals_path)

noisy_node_signals = load_dict(noisy_positions_path)

# Preparation

## Boundaries and laplacians loading

In [31]:
max_simplex_dim = 2
device = 'cuda' if torch.cuda.is_available() else 'cpu'

components = {}
components['lap'] = [coo2tensor(normalize(laplacians[i], half_interval=True)).to(device) for i in range(max_simplex_dim+1)] 

Bt_s = [boundaries[i].transpose() for i in range(max_simplex_dim)]
Bs = [boundaries[i] for i in range(max_simplex_dim)]

To have $\mathbf{B}$ start at $1$ as in the equations

In [32]:
Bt_s = [None] + Bt_s
Bs = [None] + Bs

In [33]:
num_simplices = [ L.shape[0] for L in components['lap'] ]
print(num_simplices)

[5647, 16332, 10688]


## Orthogonal decomposition
\begin{align}
    \mathbf{L}_0 =& \mathbf{B}_1 \mathbf{B}_1^\top \\
    \mathbf{L}_k =& \mathbf{B}_k^\top \mathbf{B}_k + \mathbf{B}_{k+1}\mathbf{B}^\top_{k+1} \\ 
    \mathbf{L}_K =& \mathbf{B}_K^\top \mathbf{B}_K
\end{align}

In the following examples we will consider *max_simplex_dim* to be 2, i.e. we are considering only simplices up to triangles;

### Harmonic component
$$
    \mathbf{U_{har}} = [ker(L_0), \quad ker(L_1), \quad ker(L_2)] 
$$


In [34]:
components['har'] = [None for L in components['lap']]

### Irrotational component
$$
    \mathbf{L_{irr}} = [ B_1 B_1^\top, \quad B_2 B_2^\top, \quad None]
$$

In [35]:
components['irr'] = [None for i in range(max_simplex_dim)]

for k in range(max_simplex_dim):

    Btk_upper = Bt_s[k+1].todense()
    Bk_upper = Bs[k+1].todense()
    
    BBt = Bk_upper @ Btk_upper

    components['irr'][k] = coo_matrix(BBt)

### Solenoidal component
$$ 
    \mathbf{U_{sol}} = [ \text{None}, \quad B_1^\top B_1, \quad B_2^\top B_2 ]
$$

In [36]:
components['sol'] = [None for i in range(max_simplex_dim+1)]

for k in range(1, max_simplex_dim+1):

    Btk = Bt_s[k].todense()
    Bk = Bs[k].todense()

    BtB = Btk @ Bk
    
    components['sol'][k] = coo_matrix(BtB)

### Normalize the Laplacians
(!!) is it correct to do this?

In [37]:
components['sol'] = [coo2tensor(normalize(components['sol'][i], half_interval=True)).cuda() if components['sol'][i] != None else None for i in range(0, max_simplex_dim+1) ] 
components['irr'] = [coo2tensor(normalize(components['irr'][i], half_interval=True)).cuda() if components['irr'][i] != None else None for i in range(0, max_simplex_dim) ] 

### Check

The first irrotational component must be equal to the node Laplacian 

In [38]:
tol = 1e-5
num_nodes = len(components['irr'][0])

comparison = np.abs(components['irr'][0].cpu().to_dense() - components['lap'][0].cpu().to_dense()) <= tol 
assert comparison.all()

The sum of the first solenoidal component and the first irrotational component must be equal to the edge Laplacian. 

This does not happen as both solenoidal and irrotational components are normalized separately



In [39]:
print(components['lap'][1].to_dense())
print(components['lap'][1].shape)
res = components['irr'][1] + components['sol'][1]

print(res.to_dense())
print(res.shape)

tensor([[0.4048, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4048, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.4048,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.4048, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4048, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.4048]],
       device='cuda:0')
torch.Size([16332, 16332])
tensor([[ 0.5368, -0.0660,  0.0660,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0660,  0.5368, -0.0660,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0660, -0.0660,  0.5368,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5368,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.5368,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.5368]],
       device='cuda:0')
torch.Size([16332, 16332])


# Signal preparation

## Input

In [40]:
inputs = {0 : [], 1: [], 2: []}

In [41]:
node_signal = noisy_node_signals if USE_NOISY else signals[0]
node_signal = [torch.tensor(signal) for signal in list(signals[0].values())]
node_signal = torch.stack(node_signal)
inputs[0] = node_signal

print(node_signal.shape)

torch.Size([5647, 3])


### Lift node signal to edges

In [42]:
edge_signal = Bs[1].T @ node_signal
print(edge_signal.shape)

(16332, 3)


In [43]:
print(edge_signal)
inputs[1] = torch.tensor(edge_signal)

[[ 0.024854   0.000808   0.033069 ]
 [ 0.018577  -0.0207051 -0.014109 ]
 [-0.006277  -0.0215131 -0.047178 ]
 ...
 [ 0.005298  -0.023554  -0.019772 ]
 [ 0.005297   0.023553   0.019772 ]
 [ 0.005297   0.023553  -0.019772 ]]


## Create triangle signal

In [44]:
num_triangles = Bs[2].shape[1]
triangle_signal = torch.rand((num_triangles, 3))
inputs[2] = triangle_signal

In [45]:
targets = [torch.tensor(signal) for signal in list(signals[2].values())]
targets = torch.stack(targets)

targets = targets.to(device).float()

In [46]:
inputs = list(inputs.values())
inputs = [input.transpose(1, 0) for input in inputs]

# Training

In [55]:
Bs = [ coo2tensor(B).to(device) for B in Bs if B is not None]
Bt_s = [ coo2tensor(Bt).to(device) for Bt in Bt_s if Bt is not None]

## Functions

### Train

In [113]:
def train(model, num_epochs, components, inputs, targets, Bs, Bts, optimizer, device, verbose=False):

    for epoch in tqdm(range(0, num_epochs)):

        # (max_simplex_dim+1, 1, num_simplices_dim_k)
        xs = [input.clone().to(device) for input in inputs]
        
        max_simplex_dim = len(xs)-1

        optimizer.zero_grad()
           
        # ys = model(xs, components, Bs, Bt_s)
        ys = model(xs[0])

        targets = targets.to(torch.float64)

        loss = torch.tensor(0., device=device, dtype=torch.float64)

        for i in range(3):
            loss += criterion(ys[:, i], targets[:, i])

        if verbose:
            print(f'Epoch: {epoch}, loss: {round(loss.item(), 4)}')

        loss.backward()
        optimizer.step()

## Params

In [92]:
learning_rate = 1e-3
criterion = nn.MSELoss(reduction="mean")
filter_size = 30

## Loop

In [93]:
model = MySCNN(filter_size, colors=3).to(device)

In [94]:
model = LinearBaseline(num_nodes, num_triangles).to(device)

In [ ]:
model = TopologyAwareBaseline(num_nodes, num_triangles, node_triangle_adj).to(device)

In [ ]:
print_number_of_parameters(model)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [143]:
num_epochs = 400

train(model, num_epochs, components, inputs, targets, Bs, Bt_s, optimizer, device, verbose=True)

Epoch: 0, loss: 1.2879


RuntimeError: ignored

# Evaluation

In [ ]:
xs = [input.clone().to(device) for input in inputs]
# ys = model(xs, components, Bs, Bt_s)
ys = model(xs[0])
num_triangles = inputs[2].shape[1]
assert num_triangles == len(ys)

In [ ]:
def normalize_vector(v):
    return v/torch.sqrt(v[0]**2 + v[1]**2 + v[2]**2)

In [ ]:
x_diffs, y_diffs, z_diffs = 0, 0, 0
arcs = []
predicted_norms = []

for i in range(num_triangles):
    x_diff, y_diff, z_diff = abs(ys[i] - targets[i])
    y_normalized = normalize_vector(ys[i])
    angle = y_normalized.float() @ targets[i]
    arc = np.arccos(angle.cpu().detach().numpy())/np.pi * 180
    arcs.append(arc)
    predicted_norms.append(y_normalized)
    x_diffs += x_diff
    y_diffs += y_diff
    z_diffs += z_diff


print(f'Average differences:')
print(f'\tx: {(x_diffs/num_triangles).item()}')
print(f'\ty: {(y_diffs/num_triangles).item()}')
print(f'\tz: {(z_diffs/num_triangles).item()}')
print(f'average angle: {np.mean(arcs)}')

Average differences:
	x: 0.0021703728166460583
	y: 0.001372144985654256
	z: 0.001812675663767596
average angle: 0.15855544017477466


In [ ]:
positions = original_positions

In [ ]:
stacked_triangles = np.stack(triangles)

In [ ]:
eval_targets = targets.cpu().numpy()

In [ ]:
norm_targets = 255*(eval_targets - np.min(eval_targets))/np.ptp(eval_targets)  
norm_targets = list(norm_targets)

In [ ]:
norm_targets_colors = [ f'rgb({x}, {y}, {z})' for x, y, z in norm_targets]

In [ ]:
import plotly.figure_factory as ff

import numpy as np
from scipy.spatial import Delaunay

simplices = stacked_triangles

print(simplices)
x = positions[:, 0]
y = positions[:, 1]
z = positions[:, 2]

fig = ff.create_trisurf(x=x, y=y, z=z,
                         simplices=simplices,
                         title="True normals", aspectratio=dict(x=1, y=1, z=0.3), color_func=norm_targets_colors)

fig.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
predicted_norms = [pred_norm.detach().cpu().numpy() for pred_norm in predicted_norms]

In [ ]:
predicted_norms = 255*(predicted_norms - np.min(predicted_norms))/np.ptp(predicted_norms)  
predicted_norms = list(predicted_norms)

In [ ]:
predicted_norm_colors = [ f'rgb({x}, {y}, {z})' for x, y, z in predicted_norms]

In [ ]:
fig = ff.create_trisurf(x=x, y=y, z=z,
                         simplices=simplices,
                         title="Predicted normals", aspectratio=dict(x=1, y=1, z=0.3), color_func=predicted_norm_colors)

fig.show()

Output hidden; open in https://colab.research.google.com to view.